Cambia el color del pelo

In [ ]:
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions
import cv2
import numpy as np
import time

# Configuración del ImageSegmenter
options = vision.ImageSegmenterOptions(
    base_options=BaseOptions(model_asset_path="models/hair_segmenter.tflite"),
    output_category_mask=True,
    running_mode=vision.RunningMode.LIVE_STREAM, 
    result_callback=lambda result, output_image, timestamp_ms: setattr(segmenter, 'last_result', result)
)
segmenter = vision.ImageSegmenter.create_from_options(options)

# Leer el video de entrada
cap = cv2.VideoCapture(0)
prev_time = 0
fps_limit = 15  # Limitar a 15 FPS para procesamiento

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Reducción de resolución para mejorar velocidad de procesamiento
    frame_resized = cv2.resize(frame, (640, 480))  
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    frame_rgb = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # Controlamos tasa de frames
    current_time = time.time()
    if (current_time - prev_time) * 1000 >= (1000 / fps_limit):  
        segmenter.segment_async(frame_rgb, time.time_ns() // 1_000_000)
        prev_time = current_time

    # Procesar y mostrar resultados si existen
    if hasattr(segmenter, 'last_result') and segmenter.last_result:
        segmentation_result = segmenter.last_result
        category_mask = segmentation_result.category_mask.numpy_view()

        # Crear máscara de color solo en la región de cabello
        hair_color = (255, 255, 255)  # Blanco en BGR
        color_mask = np.zeros_like(frame_resized)
        color_mask[category_mask == 1] = hair_color

        # Aplicamos un desenfoque a la máscara para un efecto más suave
        blurred_mask = cv2.GaussianBlur(color_mask, (15, 15), 0)

        # Mezclamos de imagen original con la máscara de color para resaltar el cabello
        final_frame = cv2.addWeighted(frame_resized, 1, blurred_mask, 0.4, 0)

        # Visualizar el resultado final
        cv2.imshow("Hair Color Segmentation - White Effect", final_frame)

    # Salir con la tecla 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 